Below code comes from https://github.com/jnyborg/timematch

Trial Output

In [1]:
import torch
import numpy as np
import torch.nn as nn
import json
from torchvision.transforms import transforms
from utils import RandomSamplePixels, Normalize, ToTensor, PixelSetData,\
    split_dict_train_test, pad_sequences_collate_fn, LinearLayer, PixelSetEncoder,\
        PositionalEncoding, TimeSeriesTransformer, SimpleMLP
from collections import Counter
from torch.utils.data import DataLoader

LinearLayer

In [2]:
# pixels_tmp = pixel_dataset[1901]['pixels'].permute(0, 2, 1)
pixels_tmp = torch.randn(52, 10, 32).permute(0, 2, 1)
print("input", pixels_tmp.shape)

ll = LinearLayer(10, 64)

print(ll(pixels_tmp).shape)

input torch.Size([52, 32, 10])
torch.Size([52, 32, 64])


Δοκιμή του LinearLayer με τυχαία διανύσματα όμως ΧΩΡΙΣ μάσκες

In [3]:
# mlp1_dim=[10, 32, 64]
mlp1_dim=[10, 32]
mlp2_dim=[64, 128]

layers = []
for i in range(len(mlp1_dim) - 1):
    layers.append(LinearLayer(mlp1_dim[i], mlp1_dim[i + 1]))
mlp1 = nn.Sequential(*layers)
print(mlp1)

layers = []
for i in range(len(mlp2_dim) - 1):
    layers.append(LinearLayer(mlp2_dim[i], mlp2_dim[i + 1]))
mlp2 = nn.Sequential(*layers)
print(mlp2)

# out = pixel_dataset[1901]['pixels'].unsqueeze(0)
out = torch.randn(52, 10, 32).unsqueeze(0)
print(out.shape) # torch.Size([1, 52, 10, 32])

batch, temp = out.shape[:2]
# print(batch, temp) # 1 52

out = out.view(batch * temp, *out.shape[2:]).transpose(1, 2)  # (B*T, S, C)
print(out.shape) # torch.Size([52, 32, 10])

out = mlp1(out).transpose(1, 2)
print(out.shape) # torch.Size([52, 32, 32])

out = torch.cat(
            [out.mean(dim=-1), out.std(dim=-1)],
            dim=1)
print(out.shape) # torch.Size([52, 64])

out = mlp2(out)
print(out.shape) # torch.Size([52, 128])

out = out.view(batch, temp, -1)
print(out.shape) # torch.Size([1, 52, 128])

Sequential(
  (0): LinearLayer(
    (linear): Linear(in_features=10, out_features=32, bias=False)
    (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
)
Sequential(
  (0): LinearLayer(
    (linear): Linear(in_features=64, out_features=128, bias=False)
    (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
)
torch.Size([1, 52, 10, 32])
torch.Size([52, 32, 10])
torch.Size([52, 32, 32])
torch.Size([52, 64])
torch.Size([52, 128])
torch.Size([1, 52, 128])


## PSE Part

Οι extra χωρικές πληροφορίες δε χρειάζονται

Είσοδος και έξοδος PSE πριν προστεθεί η μάσκα

In [4]:
# # input = pixel_dataset[1911]['pixels'].unsqueeze(0)
# input = torch.randn(52, 10, 32).unsqueeze(0)
# print(input.shape)
# out = pixel_set_encoder(input)
# print(out.shape) # torch.Size([1, 52, 128])

Δημιουργία Dataset και Dataloaders για τη δοκιμή του PSE και του Transformer Encoder

In [5]:
# labels_200 will contain the labels with more than 200 occurrences
f_labels = open(r"Exercise4\timematch_data\denmark\32VNH\2017\meta\labels_cleaned.json")
labels_200 = json.load(f_labels)

print(len(labels_200), len(set(labels_200.values())))

count = 0
for lab in labels_200:
    print(lab, labels_200[lab])
    count += 1
    if count == 6:
        break

labels_200_counter = Counter(labels_200.values())

print()

for lab in labels_200_counter:
    print(lab, labels_200_counter[lab])

4255 7
0 corn
1 corn
2 corn
3 corn
4 corn
5 spring_barley

corn 275
spring_barley 1141
meadow 1013
winter_wheat 856
winter_rapeseed 301
winter_barley 352
winter_rye 317


Οι παρακάτω αντιστοιχίσεις είναι διαφορετικές από του dataset_creation αλλά δεν πειράζει

In [6]:
class_to_idx = {cls: idx for idx, cls in enumerate(labels_200_counter)}
for key, val in class_to_idx.items():
    print(key, val)

print()

train_labels, val_labels = split_dict_train_test(labels_200, test_size=0.2)

print(len(labels_200))
print(len(train_labels))
print(len(val_labels))

corn 0
spring_barley 1
meadow 2
winter_wheat 3
winter_rapeseed 4
winter_barley 5
winter_rye 6

4255
3400
855


In [7]:
train_transform = transforms.Compose([RandomSamplePixels(32), Normalize(), ToTensor()])

test_transform = transforms.Compose([Normalize(), ToTensor()])

train_dataset = PixelSetData("Exercise4/timematch_data/denmark/32VNH/2017",
                             class_to_idx, train_labels,
                             train_transform)
print(len(train_dataset))
print()
val_dataset = PixelSetData("Exercise4/timematch_data/denmark/32VNH/2017",
                             class_to_idx, val_labels,
                             test_transform)
print(len(val_dataset))

('Exercise4/timematch_data/denmark/32VNH/2017\\data\\0.zarr', '0', 'corn')


('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1.zarr', '1', 'corn')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\10.zarr', '10', 'corn')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\100.zarr', '100', 'winter_rapeseed')
3400

('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1003.zarr', '1003', 'winter_barley')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1009.zarr', '1009', 'winter_barley')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1014.zarr', '1014', 'meadow')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1017.zarr', '1017', 'meadow')
855


In [8]:
train_dloader = DataLoader(
    train_dataset, batch_size=8, shuffle=True,
    collate_fn=pad_sequences_collate_fn,
    # num_workers=4,
)
val_dloader = DataLoader(
    val_dataset, batch_size=8, shuffle=True,
    collate_fn=pad_sequences_collate_fn,
)

print(len(train_dloader))
print(len(val_dloader))

for batch in train_dloader:
    print(batch[0].shape)
    print(batch[1].shape)
    print(batch[1])
    print(batch[2].shape)
    print(np.unique(batch[2]))
    print()
    break

for batch in val_dloader:
    print(batch[0].shape)
    print(batch[1].shape)
    print(batch[1])
    print(batch[2].shape)
    print(np.unique(batch[2]))
    print()
    break

425
107
torch.Size([8, 52, 10, 32])
torch.Size([8])
tensor([6, 0, 1, 4, 1, 1, 5, 1])
torch.Size([8, 52, 32])
[ True]

torch.Size([8, 52, 10, 2872])
torch.Size([8])
tensor([1, 5, 2, 2, 2, 1, 1, 4])
torch.Size([8, 52, 2872])
[False  True]



In [9]:
pixel_set_encoder = PixelSetEncoder(10, mlp1=[10, 32],
                                    pooling="mean_std",
                                    mlp2=[64, 128])
print(pixel_set_encoder)
print()

for batch in train_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(mask.shape)

    break

print()

out = pixel_set_encoder(input_tensor, mask)
print(out.shape)

print()

print(np.unique(out.detach().numpy()))

PixelSetEncoder(
  (mlp1): Sequential(
    (0): LinearLayer(
      (linear): Linear(in_features=10, out_features=32, bias=False)
      (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
  )
  (mlp2): Sequential(
    (0): LinearLayer(
      (linear): Linear(in_features=64, out_features=128, bias=False)
      (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
  )
)

torch.Size([8, 52, 10, 32])
torch.Size([8])
torch.Size([8, 52, 32])

torch.Size([8, 52, 128])

[0.0000000e+00 2.9236078e-05 3.0653551e-05 ... 5.7846966e+00 6.2438588e+00
 6.3338661e+00]


Without collate_fn=pad_sequences_collate_fn

In [10]:
# train_dloader = DataLoader(
#     train_dataset,
#     batch_size=8,
#     shuffle=True,
#     # collate_fn=pad_sequences_collate_fn,
#     # num_workers=4,
# )

# print(len(train_dloader))

# for batch in train_dloader:
#     input_tensor = batch["pixels"]
#     labels = batch["label_idx"]
#     mask = batch["valid_pixels"]

#     print(input_tensor.shape)
#     print(labels.shape)
#     print(labels)
#     print(mask.shape)
#     print()

#     break

# out = pixel_set_encoder(input_tensor, mask)

# print(out.shape)

# print(np.unique(out.detach().numpy()))
# print(out)

## Transformer Part

In [18]:
# can also check Tsironis
pe = PositionalEncoding(128)

input = torch.randn(1, 52, 128)
out = pe(input)

print(out.shape)

PositionalEncoding()
torch.Size([1, 52, 128])


In [12]:
# time_series_transformer = TimeSeriesTransformer(feature_size=128)
time_series_transformer = TimeSeriesTransformer(d_model=128)
print(time_series_transformer)

input = torch.randn(1, 52, 128)
out = time_series_transformer(input)
print("Output shape: ", out.shape)

TimeSeriesTransformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (pos_emb): PositionalEncoding()
)
Output shape:  torch.Size([1, 128])


In [13]:
simple_mlp = SimpleMLP(input_dim=128, output_dim=7)
print(simple_mlp)

# input = torch.randn(1, 128)
input = out
out = simple_mlp(input)

print(out.shape)

print(out)

SimpleMLP(
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=7, bias=True)
  )
)
torch.Size([1, 7])
tensor([[ 0.0175,  0.0439, -0.0170,  0.1822, -0.1093, -0.4548, -0.0627]],
       grad_fn=<AddmmBackward0>)


In [14]:
# # pixel_encoder = PixelSetEncoder(input_dim=32, hidden_dim=64, output_dim=128)
pixel_encoder = PixelSetEncoder(10, mlp1=[10, 32],
                                pooling="mean_std",
                                mlp2=[64, 128])

# transformer_encoder = TimeSeriesTransformer(feature_size=128)
transformer_encoder = TimeSeriesTransformer(d_model=128)

simple_mlp = SimpleMLP(input_dim=128, output_dim=7)

# before adding mask
# output = model(input_tensor)

In [15]:
class CompleteModel(nn.Module):
    def __init__(self, pixel_encoder, transformer_encoder, simple_mlp):
        super(CompleteModel, self).__init__()
        self.pixel_encoder = pixel_encoder
        self.transformer_encoder = transformer_encoder
        self.simple_mlp = simple_mlp
    
    def forward(self, x, mask):
        x = self.pixel_encoder(x, mask)  # torch.Size([8, 52, 128])
        x = self.transformer_encoder(x)  # torch.Size([8, 128])
        x = self.simple_mlp(x) # torch.Size([8, 7])
        return x

model = CompleteModel(pixel_encoder, transformer_encoder, simple_mlp)

Δοκιμή Ολόκληρου Μοντέλου για είσοδο από Train Dataset με σταθερό πλήθος pixels = 32

In [16]:
train_dloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=pad_sequences_collate_fn,
    # num_workers=4,
)

for batch in train_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(labels)
    print(mask.shape)
    print()

    break

# out = pixel_encoder(input_tensor, mask)
# out = transformer_encoder(out)
# out = simple_mlp(out)

out = model(input_tensor, mask)

print(out.shape)

print(out)

torch.Size([8, 52, 10, 32])
torch.Size([8])
tensor([1, 0, 3, 1, 3, 1, 2, 2])
torch.Size([8, 52, 32])

torch.Size([8, 7])
tensor([[-0.1727,  0.0470, -0.0699, -0.2945,  0.2160,  0.3588, -0.0322],
        [-0.1711, -0.0410, -0.1523, -0.0953,  0.1853,  0.4787,  0.1776],
        [-0.1915,  0.1050, -0.2368, -0.0611,  0.0703,  0.3974,  0.0005],
        [-0.1398,  0.0335, -0.1930, -0.1196,  0.1740,  0.3175, -0.0538],
        [-0.0662,  0.1261, -0.1321,  0.0670,  0.1845,  0.2747, -0.0215],
        [-0.1758,  0.1575, -0.1197,  0.1476,  0.2580,  0.3085, -0.0586],
        [-0.0270, -0.1638, -0.1319, -0.0501,  0.1577,  0.3406, -0.0881],
        [-0.1372, -0.0958, -0.1480, -0.0944,  0.2829,  0.2929,  0.2447]],
       grad_fn=<AddmmBackward0>)


Δοκιμή Ολόκληρου Μοντέλου για είσοδο από Val Dataset με μεταβλητό πλήθος pixels

In [17]:
for batch in val_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(labels)
    print(mask.shape)
    print()

    break

# out = pixel_encoder(input_tensor, mask)
# out = transformer_encoder(out)
# out = simple_mlp(out)

out = model(input_tensor, mask)

print(out.shape)

print(out)

torch.Size([8, 52, 10, 481])
torch.Size([8])
tensor([3, 2, 1, 2, 2, 1, 3, 2])
torch.Size([8, 52, 481])

torch.Size([8, 7])
tensor([[-0.2786, -0.0051, -0.0927, -0.2523,  0.0856,  0.3379,  0.0127],
        [-0.1772,  0.0250, -0.1623, -0.1980,  0.3534,  0.3684,  0.1281],
        [-0.1173,  0.0588, -0.1673, -0.0185,  0.0347,  0.4545,  0.1378],
        [-0.1696,  0.1196, -0.1855,  0.0449,  0.0979,  0.4006,  0.1338],
        [-0.2636, -0.0120, -0.2221,  0.0861,  0.2160,  0.3663,  0.0974],
        [-0.0561, -0.1237, -0.1480,  0.0695,  0.3556,  0.5386,  0.0969],
        [-0.0051,  0.0267, -0.2549, -0.1177,  0.2214,  0.3052,  0.0976],
        [-0.1395,  0.0360, -0.1237, -0.1323,  0.2120,  0.3655,  0.1363]],
       grad_fn=<AddmmBackward0>)
